# U.S. Border Entry ETL
---

## Extract

U.S. border-crossing data was extracted from the Bureau of Transportation Statistics (BTS) Border Crossing API ( https://data.transportation.gov/Research-and-Statistics/Border-Crossing-Entry-Data/keg4-3bc2 ). Seeing as this data is found in the Socrata Public Data API, instead of extracting border-crossing data via JSON, the sodapy library was used.

In [50]:
# Dependencies
import pandas as pd
from sodapy import Socrata
from sqlalchemy import create_engine

In [43]:
# Activate the Socrata Public Data API, specifically transportation data
client = Socrata("data.transportation.gov", None)

In order to ensure that the throttling limits were not reached, conditions were added when extracting the data.

In [3]:
# Create conditions to ensure most rows will be used in final database
conditions = "date >= '2008-01-01' and value > 0 and (measure = 'Personal Vehicles' or \
              measure = 'Personal Vehicles Passengers' or measure = 'Bus Passengers' or measure = 'Train Passengers') "

In [7]:
# Request Border Entry data from specified API including additional conditions
results = client.get("keg4-3bc2", limit = 23000,  border = 'US-Canada Border', where = conditions)

In [8]:
# Create a Pandas Dataframe
df = pd.DataFrame.from_records(results)

In [45]:
# Sort the Dataframe by date
df = border_entry_df.sort_values(by=['date'])

## Transform

Columns such as border and port_id were removed since the border column only had one value "US-Canada Border" and port_id was not useful for additional analysis. The date column was then formatted to a datetime object for better readibility. Columns were renamed for user clarity and the index was reset.

In [80]:
# Create a copy of the dataframe to manipulate the columns
border_df = df.copy()
# Remove border and port code columns
border_df = border_df[['date','measure','port_name','state','value']]
# Change data type for date column
border_df['date'] = pd.to_datetime(border_df['date'])
# Rename columns accordingly
border_df = border_df.rename(columns={"date": "Date", "measure": "Transportation Type", "port_name": "City", 
                                      "state": "State", "value": "Number of Passengers"})
# Reset index to start at 0
border_df = border_df.reset_index(drop=True)

## Load

A connection was made to the border_db database in Postgres. A table called border_entry was created (see border_entry_schema.sql for completed schema). Using pandas, the dataframe was loaded into the border_db.

In [85]:
# Connect to local database
connection_string = "postgres: @localhost:5432/border_db"
engine = create_engine(f'postgresql://{connection_string}')

In [87]:
# Check for tables
engine.table_names()

['border_entry']

In [89]:
# Use pandas to load Dataframe into the database
border_df.to_sql(name='border_entry', con=engine, if_exists='append', index=False)

In [92]:
# Confirm that the data was added by querying the database
pd.read_sql_query('select * from border_entry', con=engine).head()

,ID,Date,Transportation Type,City,State,Number of Passengers
0,1,2008-01-01,Personal Vehicles,Oroville,Washington,14091
1,2,2008-01-01,Bus Passengers,Eastport,Idaho,227
2,3,2008-01-01,Personal Vehicles,Limestone,Maine,3030
3,4,2008-01-01,Bus Passengers,Calais,Maine,405
4,5,2008-01-01,Bus Passengers,Metaline Falls,Washington,183
